In [1]:
import streamlit as st
from campaign_generator import CampaignGenerator

In [2]:
%matplotlib inline

In [3]:
campaign_details = "D&D5e in many of the popular settings, each different than the last. The campaign begins with a small unassuming conflict which is connected to a larger conflict that the players uncover as the progress through the campaign."

In [4]:
generator = CampaignGenerator(
                base_url="http://localhost:11434",
                model="mistral",
            )

In [5]:
players ="""name: Eilif Stoneheart,
        backstory: His family was killed by undead. Seeking vengeance, he joins forces with a group of adventurers who can help him destroy the source of these monstrosities.
        name: "Sylva Oakhollow,
        backstory: Her village was drained of its life force by Xaviar. Seeking to save her people, she teams up with others who have also lost loved ones.
        name: "Thalia Moonshadow,
        backstory: A former member of the cult, she seeks redemption for her past actions and wants to stop Xaviar from unleashing the undead upon the world."""

In [6]:
generator.generate_plot(campaign_details, players)

Generating plot...
 {
     "plot": {
       "main_quest": "The players must uncover the truth about Xaviar, a powerful necromancer who is using an ancient artifact to create undead armies and drain life forces from living beings.",
       "key_locations": [
         { "name": "Ironpeak Mines", "description": "A series of abandoned mines rumored to hold the key to stopping Xaviar's necromantic rituals." },
         { "name": "Whisperwind Forest", "description": "An enchanted forest where Xaviar is believed to be hiding his cult and the artifact he uses for his dark deeds." },
         { "name": "The Shrouded Citadel", "description": "Xaviar's stronghold, the final showdown with Xaviar will take place here." }
       ],
       "act_one": {
         "summary": "The players investigate the Ironpeak Mines and learn about Xaviar's necromantic rituals. They also discover a prophecy that foretells the rise of an undead army.",
         "checkpoints": [
           { "name": "Investigate the Iro

' {\n     "plot": {\n       "main_quest": "The players must uncover the truth about Xaviar, a powerful necromancer who is using an ancient artifact to create undead armies and drain life forces from living beings.",\n       "key_locations": [\n         { "name": "Ironpeak Mines", "description": "A series of abandoned mines rumored to hold the key to stopping Xaviar\'s necromantic rituals." },\n         { "name": "Whisperwind Forest", "description": "An enchanted forest where Xaviar is believed to be hiding his cult and the artifact he uses for his dark deeds." },\n         { "name": "The Shrouded Citadel", "description": "Xaviar\'s stronghold, the final showdown with Xaviar will take place here." }\n       ],\n       "act_one": {\n         "summary": "The players investigate the Ironpeak Mines and learn about Xaviar\'s necromantic rituals. They also discover a prophecy that foretells the rise of an undead army.",\n         "checkpoints": [\n           { "name": "Investigate the Ironpea

In [1]:
#campaign = generator.generate_campaign(campaign_details)

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from typing import TypedDict, Dict, List, Any
from langchain_ollama import OllamaLLM
import toml

class DungeonMasterAgent:

    def __init__(self, base_url: str, model: str):
        self.base_url = base_url
        self.model = model
        self.llm = self.initialize_llm()
        self.prompts = self.load_prompts_from_file("prompts.toml")
        self.game_graph = self.create_game_graph()

    def load_prompts_from_file(self, file_path: str) -> Dict[str, str]:
        try:
            with open(file_path, "r") as file:
                return toml.load(file)
        except Exception as e:
            print(f"Error loading prompts from {file_path}: {e}")
            return {}

    def initialize_llm(self):
        try:
            return OllamaLLM(base_url=self.base_url, model=self.model, max_tokens=2048, temperature=0.7)
        except Exception as e:
            print(f"error: {e}")
            return None

    def create_game_graph(self):
        # Define the state schema
        class GameState(TypedDict):
            context: str
            user_prompt: str
            intent: str
            response: str
            history: List[Dict[str, str]]

        def set_tools(state: GameState):
            # Logic to set tools based on the game state
            state["tools"] = ["recognize_intent"]

        # Create nodes for your graph
        def recognize_intent(state: GameState):
            # Logic to recognize intent using your existing intent agent
            response = self.llm.invoke(str.format(self.prompts["intent_recognition"], state['tools'], state['user_prompt']))
            state["intent"] = response
            return state

        def handle_dm_question(state: GameState):
            # Logic to handle DM questions
            state["response"] = "DM response"
            return state

        def handle_action(state: GameState):
            # Logic to handle character actions
            state["response"] = "Action result"
            return state

        # Define the graph
        game_graph = StateGraph(GameState)

        # Add nodes
        game_graph.add_node("intent_recognition", recognize_intent)
        game_graph.add_node("dm_question", handle_dm_question)
        game_graph.add_node("character_action", handle_action)

        # Add edges
        game_graph.add_edge("intent_recognition", "dm_question")
        game_graph.add_conditional_edges(
            "intent_recognition",
            lambda state: state["recognized_intent"],
            {
                "ask_dm": "dm_question",
                "character_action": "character_action"
            }
        )

        # Set starting node
        game_graph.set_entry_point("intent_recognition")

        # Edges from result nodes to END
        game_graph.add_edge("dm_question", END)
        game_graph.add_edge("character_action", END)

        # Compile the graph
        return game_graph.compile()

    def run_game(self, initial_state: Dict[str, Any]):
        return self.game_graph.run(initial_state)

In [5]:
dm = DungeonMasterAgent(base_url="http://localhost:11434", model="mistral")

In [6]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

In [7]:
class AgentState(TypedDict):
    """The state of the agent."""

    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [8]:
from langchain_core.tools import tool

In [15]:
from langchain_core.tools import tool
from langchain_ollama.chat_models import ChatOllama
model = ChatOllama(base_url="http://localhost:11434", model="mistral", max_tokens=2048, temperature=0.7)

@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    # Don't let the LLM know this though 😊
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]

model = model.bind_tools(tools)

In [17]:
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig

tools_by_name = {tool.name: tool for tool in tools}


# Define our tool node
def tool_node(state: AgentState):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# Define the node that calls the model
def call_model(
    state: AgentState,
    config: RunnableConfig,
):
    # this is similar to customizing the create_react_agent with 'prompt' parameter, but is more flexible
    system_prompt = SystemMessage(
        "You are a helpful AI assistant, please respond to the users query to the best of your ability!"
    )
    response = model.invoke([system_prompt] + state["messages"], config)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the conditional edge that determines whether to continue or not
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"